In [1]:
# Importing packages
import requests
import json
import time #will use this to pause execution for a few seconds

import numpy as np
import pandas as pd

from pandas import json_normalize

In [2]:
# store API key
api_key = YOUR_API_KEY

In [3]:
# The initial string for the api reuqest
endpoint_url_start = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

In [4]:
# Creating dataframes to be filled
hilo_common = pd.DataFrame(columns=['name','place_id', 'address','coordinates',\
                                    'business_status', 'types', 'delivery', 'dine_in',\
                                    'takeout', 'price_level', 'rating', 'user_ratings_total',\
                                    'serves_breakfast', 'serves_brunch','serves_lunch',\
                                    'serves_dinner', 'serves_vegetarian_food', 'serves_beer',
                                    'serves_wine'])
hilo_uncommon = pd.DataFrame(columns=['name','place_id', 'address','coordinates',\
                                    'business_status', 'types', 'delivery', 'dine_in',\
                                    'takeout', 'price_level', 'rating', 'user_ratings_total',\
                                    'serves_breakfast', 'serves_brunch','serves_lunch',\
                                    'serves_dinner', 'serves_vegetarian_food', 'serves_beer',
                                    'serves_wine'])

In [5]:
# Specifying the types of places in lists
# common = multiple small radii for api search
# uncommon = whole city radius search
places_common = ['bakery', 'cafe', 'restaurant', 'meal_delivery', 'meal_takeaway', 'bar', 'convenience store',\
                 'gas_station', 'supermarket', 'lodging', 'department_store', 'tourist_attraction']
places_uncommon = ['airport', 'amusement_park', 'aquarium', 'casino', 'hospital',\
                   'liquor_store', 'stadium', 'university', 'zoo', 'movie_theater', 'primary_school',\
                   'school', 'secondary_school', 'shopping mall', 'bowling_alley']

In [6]:
# uncommon/large radius search info
central_cord = "19.68933, -155.08268"
central_radius = "9500"

In [7]:
# common/small radius searches info
cords = ["19.71890,-155.09343", "19.72473,-155.06600", "19.72600,-155.02977", "19.70562,-155.08258",\
         "19.70661,-155.05612", "19.69671,-155.11744", "19.67937,-155.07814", "19.67890,-155.03994",\
         "19.68336,-155.16312", "19.64372,-155.11469"]
radii = ["1500", "1500", "2500", "1500",\
         "1500", "2500", "2500", "3000",\
         "3000", "4000"]

In [8]:
# function to append result to the dataframe
# Takes care of null returned from API request
def append_place(json_results, df):
    l = len(df)
    for i, val in enumerate(json_results['results']):
        df.loc[l + i] = [val.get('name', np.nan), 
                         val.get('place_id', np.nan),
                         val.get('vicinity', np.nan), 
                         val.get('geometry', np.nan).get('location', np.nan),
                         val.get('business_status', np.nan),
                         val.get('types', np.nan),
                         val.get('delivery', np.nan),
                         val.get('dine_in', np.nan),
                         val.get('takeout', np.nan),
                         val.get('price_level', np.nan),
                         val.get('rating', np.nan),
                         val.get('user_ratings_total', np.nan),
                         val.get('serves_breakfast', np.nan),
                         val.get('serves_brunch', np.nan),
                         val.get('serves_lunch',np.nan),
                         val.get('serves_dinner', np.nan),
                         val.get('serves_vegetarian_food', np.nan),
                         val.get('serves_beer', np.nan),
                         val.get('serves_wine', np.nan)]

In [9]:
# creating a dataset for the uncommon places
for _type in places_uncommon:
    endpoint_url = endpoint_url_start + "&location=" + central_cord + "&radius=" + \
    central_radius + "&region=us&type=" + _type + "&key=" + api_key
    
    params = {}
    
    res = requests.get(endpoint_url, params = params)
    results =  json.loads(res.content)
    append_place(results, hilo_uncommon)
    
    
    time.sleep(2)
    
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token']
        res = requests.get(endpoint_url, params = params)
        results =  json.loads(res.content)
        append_place(results, hilo_uncommon)
        time.sleep(1)

In [10]:
# view created dataset to check it looks ok
hilo_uncommon

,Name,place_id,Address,Coordinates,business_status,types,delivery,dine_in,takeout,price_level,rating,user_ratings_total,serves_breakfast,serves_brunch,serves_lunch,serves_dinner,serves_vegetarian_food,serves_beer,serves_wine
0,Hilo International Airport,ChIJA_6Mr3hMUnkRf2xxZUX-my4,"2450 Kekuanaoa Street, Hilo","{'lat': 19.7188342, 'lng': -155.0416866}",OPERATIONAL,"[airport, point_of_interest, establishment]",NaN,NaN,NaN,NaN,4.4,776.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Blue Hawaiian Helicopters,ChIJqx06aMK0U3kR6i4j-j9NX34,United States,"{'lat': 19.7150176, 'lng': -155.0434895}",OPERATIONAL,"[airport, travel_agency, tourist_attraction, p...",NaN,NaN,NaN,NaN,4.0,194.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hilo Medical Center Heliport,ChIJeQfmUem0U3kR8UKXpxmqQfY,"1190 Waianuenue Avenue, Hilo","{'lat': 19.7184778, 'lng': -155.1143653}",OPERATIONAL,"[airport, point_of_interest, establishment]",NaN,NaN,NaN,NaN,4.5,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hilo Medical Center,ChIJ6VYwrKS0U3kR9caoLq5TmrM,"1190 Waianuenue Avenue, Hilo","{'lat': 19.7182136, 'lng': -155.11293}",OPERATIONAL,"[hospital, health, point_of_interest, establis...",NaN,NaN,NaN,NaN,3.3,121.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Straub Medical Center - Hilo Clinic,ChIJq8gDaK20U3kRmgmmNJcMFOY,"75 Puuhonu Place, Hilo","{'lat': 19.7158175, 'lng': -155.0965672}",OPERATIONAL,"[hospital, doctor, health, point_of_interest, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,"Dr. Jan L. Houser, MD",ChIJ0-Mfz7y0U3kRJRU5clpOUw4,"1292 Waianuenue Avenue, Hilo","{'lat': 19.717876, 'lng': -155.115532}",OPERATIONAL,"[doctor, point_of_interest, health, establishm...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,Associa Hawaii,ChIJq0W1Hl1LUnkRw091IqrOeAo,"400 Hualani Street #294, Hilo","{'lat': 19.7133918, 'lng': -155.0736677}",CLOSED_TEMPORARILY,"[point_of_interest, establishment]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,Hilo Bay Hostel,ChIJaTyf3qlMUnkRCudhTGi4dQA,"101 Waianuenue Avenue, Hilo","{'lat': 19.7257001, 'lng': -155.088042}",OPERATIONAL,"[lodging, point_of_interest, establishment]",NaN,NaN,NaN,NaN,3.7,172.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,Walmart,ChIJvUAjN21LUnkRVmbJGBS4VZ8,"325 Makaala Street, Hilo","{'lat': 19.70026589999999, 'lng': -155.0618477}",OPERATIONAL,"[department_store, hardware_store, grocery_or_...",NaN,NaN,NaN,1.0,3.9,2844.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# creating a dataset for the common places
for idx, circle in enumerate(cords):
    for _type in places_common:
        endpoint_url = endpoint_url_start + "&location=" + circle + "&radius=" + \
        radii[idx] + "&region=us&type=" + _type + "&key=" + api_key

        params = {}

        res = requests.get(endpoint_url, params = params)
        results =  json.loads(res.content)
        append_place(results, hilo_common)

        time.sleep(2)
        
        # Pulling results from other pages
        while "next_page_token" in results:
            params['pagetoken'] = results['next_page_token']
            res = requests.get(endpoint_url, params = params)
            results =  json.loads(res.content)
            append_place(results, hilo_common)
            time.sleep(1)

In [12]:
# view created dataset to check it looks ok
hilo_common

,Name,place_id,Address,Coordinates,business_status,types,delivery,dine_in,takeout,price_level,rating,user_ratings_total,serves_breakfast,serves_brunch,serves_lunch,serves_dinner,serves_vegetarian_food,serves_beer,serves_wine
0,Papa'a Palaoa Bakery,ChIJV30QdVdLUnkRF1oH6Bho5SU,"187 Kilauea Avenue, Hilo","{'lat': 19.7220616, 'lng': -155.085246}",OPERATIONAL,"[bakery, food, store, point_of_interest, estab...",NaN,NaN,NaN,1.0,4.6,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Low International Food,ChIJzRUOcldLUnkRuVdgapXhlEg,"222 Kilauea Avenue, Hilo","{'lat': 19.7216053, 'lng': -155.0851868}",OPERATIONAL,"[bakery, restaurant, food, store, point_of_int...",NaN,NaN,NaN,1.0,4.3,114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Moonstruck,ChIJB-pawVdLUnkR8krKiIkAgus,"16 Furneaux Lane, Hilo","{'lat': 19.7240477, 'lng': -155.0857916}",OPERATIONAL,"[bakery, food, store, point_of_interest, estab...",NaN,NaN,NaN,2.0,4.5,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nector Cafe,ChIJj2hxXbdLUnkRftSaRFVgczg,"216 Kamehameha Avenue, Hilo","{'lat': 19.7248671, 'lng': -155.086052}",OPERATIONAL,"[cafe, bakery, food, store, point_of_interest,...",NaN,NaN,NaN,NaN,4.1,143.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Grandma's Cookie Jar,ChIJZcKu36tMUnkR3tp-0kDxS-Y,Hilo,"{'lat': 19.7297152, 'lng': -155.0900154}",OPERATIONAL,"[bakery, food, store, point_of_interest, estab...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022,MTB Park,ChIJDZinPqE1UnkRY8XNd6UImrw,"JWJ9+4V, Hilo","{'lat': 19.6303622, 'lng': -155.0802767}",OPERATIONAL,"[park, point_of_interest, establishment]",NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1023,Diversified AG Products Inc,ChIJ4zOHsy61U3kRHy4RuvoUC7A,"1233 Puhau Street, Hilo","{'lat': 19.6775606, 'lng': -155.1028337}",CLOSED_TEMPORARILY,"[point_of_interest, food, establishment]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1024,Waiakea Uka Park,ChIJXfCNzzm1U3kRSBSWtmws4Z8,Hilo,"{'lat': 19.6652191, 'lng': -155.1095011}",NaN,"[park, premise, point_of_interest, establishment]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1025,Parking lot,ChIJa3yQnzC1U3kRkcD3rSldDFo,Hilo,"{'lat': 19.6687921, 'lng': -155.1067196}",NaN,"[premise, parking, point_of_interest, establis...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# creating a list of unique place ids that we can use in Google Places Details API search to get business hours
h_common_ids = np.unique(hilo_common['place_id'])
h_uncommon_ids = np.unique(hilo_uncommon['place_id'])

In [14]:
# creating dataset to fill with business hours
h_common_hours = pd.DataFrame(columns=['Name','place_id','business_hours', 'business_hours_text'])
h_uncommon_hours = pd.DataFrame(columns=['Name','place_id','business_hours','business_hours_text'])

In [15]:
# new api request string for new type of API requests
endpoint_url_start_2 = "https://maps.googleapis.com/maps/api/place/details/json?place_id="

In [16]:
# new function to append results from Places Details search to business hours dataset
def append_place_2(json_results, df):
    l = len(df)
    val = json_results['result']
    if len(val) == 2:
        df.loc[l + 1] = [val.get('name', np.nan), 
                         val.get('place_id', np.nan),
                         np.nan,
                         np.nan]
    else:
        df.loc[l + 1] = [val.get('name', np.nan), 
                         val.get('place_id', np.nan),
                         val.get('opening_hours', np.nan).get('periods', np.nan),
                         val.get('opening_hours', np.nan).get('weekday_text', np.nan)]

In [20]:
# creating a business hours dataset for the common places
for place_id in h_common_ids:
    endpoint_url = endpoint_url_start_2 + place_id + \
    "&fields=name%2Cplace_id%2Copening_hours&key=" + api_key
    
    params = {}
    
    res = requests.get(endpoint_url, params = params)
    results =  json.loads(res.content)
    append_place_2(results, h_common_hours)
    
    time.sleep(1)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800


In [29]:
# view created dataset to check it looks ok
h_common_hours

,Name,place_id,business_hours,business_hours_text
0,Hele,ChIJ-6iXTom1U3kReva095WCmEg,"[{'close': {'day': 0, 'time': '2130'}, 'open':...","[Monday: 5:30 AM – 9:30 PM, Tuesday: 5:30 AM –..."
1,Big Island Gallery,ChIJ-7bLyay1U3kRUQt_7JlOQnQ,"[{'close': {'day': 0, 'time': '1700'}, 'open':...","[Monday: 9:00 AM – 5:00 PM, Tuesday: 9:00 AM –..."
2,Pasha Group,ChIJ-93lboZMUnkR02kgE8yqxUE,NaN,NaN
3,Horizon Lines,ChIJ-9uB_ZlMUnkRhv1FL-RhcFE,NaN,NaN
4,Texaco,ChIJ-Q4oEVZLUnkRLolvOpGcpXg,"[{'close': {'day': 0, 'time': '1800'}, 'open':...","[Monday: 6:00 AM – 8:00 PM, Tuesday: 6:00 AM –..."
...,...,...,...,...
829,Inouye Electric LLC,ChIJzUm17TO1U3kR8KUb_1f1CTY,NaN,NaN
830,Signature Flight Support ITO - Hilo Internatio...,ChIJzXf_LWNLUnkRP8304CL9mZE,"[{'close': {'day': 0, 'time': '1700'}, 'open':...","[Monday: 8:00 AM – 5:00 PM, Tuesday: 8:00 AM –..."
831,Taketa Iwata Hara & Associates LLC,ChIJza4uGFxLUnkR4RQHDTdC2jA,"[{'close': {'day': 1, 'time': '1700'}, 'open':...","[Monday: 8:00 AM – 5:00 PM, Tuesday: 8:00 AM –..."
832,R T's Plumbing Inc,ChIJzxQB5bi1U3kRQz-TEfv-uLs,NaN,NaN


In [32]:
# creating a business hours dataset for the uncommon places
for place_id in h_uncommon_ids:
    endpoint_url = endpoint_url_start_2 + place_id + \
    "&fields=name%2Cplace_id%2Copening_hours&key=" + api_key
    
    params = {}
    
    res = requests.get(endpoint_url, params = params)
    results =  json.loads(res.content)
    append_place_2(results, h_uncommon_hours)
    
    
    time.sleep(1)

0
50
100
150


In [43]:
# view created dataset to check it looks ok
h_uncommon_hours

,name,place_id,business_hours,business_hours_text
0,Hele,ChIJ-6iXTom1U3kReva095WCmEg,"[{'close': {'day': 0, 'time': '2130'}, 'open':...","[Monday: 5:30 AM – 9:30 PM, Tuesday: 5:30 AM –..."
1,Big Island Gallery,ChIJ-7bLyay1U3kRUQt_7JlOQnQ,"[{'close': {'day': 0, 'time': '1700'}, 'open':...","[Monday: 9:00 AM – 5:00 PM, Tuesday: 9:00 AM –..."
2,Pasha Group,ChIJ-93lboZMUnkR02kgE8yqxUE,NaN,NaN
3,Horizon Lines,ChIJ-9uB_ZlMUnkRhv1FL-RhcFE,NaN,NaN
4,Texaco,ChIJ-Q4oEVZLUnkRLolvOpGcpXg,"[{'close': {'day': 0, 'time': '1800'}, 'open':...","[Monday: 6:00 AM – 8:00 PM, Tuesday: 6:00 AM –..."
...,...,...,...,...
829,Inouye Electric LLC,ChIJzUm17TO1U3kR8KUb_1f1CTY,NaN,NaN
830,Signature Flight Support ITO - Hilo Internatio...,ChIJzXf_LWNLUnkRP8304CL9mZE,"[{'close': {'day': 0, 'time': '1700'}, 'open':...","[Monday: 8:00 AM – 5:00 PM, Tuesday: 8:00 AM –..."
831,Taketa Iwata Hara & Associates LLC,ChIJza4uGFxLUnkR4RQHDTdC2jA,"[{'close': {'day': 1, 'time': '1700'}, 'open':...","[Monday: 8:00 AM – 5:00 PM, Tuesday: 8:00 AM –..."
832,R T's Plumbing Inc,ChIJzxQB5bi1U3kRQz-TEfv-uLs,NaN,NaN


In [54]:
# merging previous datasets with business hours datasets
hilo_common = hilo_common.merge(h_common_hours, how='left', left_on=['name', 'place_id'], right_on=['name', 'place_id'])
hilo_uncommon = hilo_uncommon.merge(h_uncommon_hours, how='left', left_on=['name', 'place_id'], right_on=['name', 'place_id'])

In [57]:
# exporting data sets to local csv files
hilo_common.to_csv('../data/hilo_common_dist.csv')
hilo_uncommon.to_csv('../data/hilo_uncommon_dist.csv')